In [31]:
from stable_baselines3 import PPO
from myosuite.utils import gym as myo_gym

from config import Config


In [32]:
cfg = Config()

env = myo_gym.make(cfg.env_id)
obs, info = env.reset()


In [33]:
action = env.action_space.sample()

obs, reward, terminated, truncated, info = env.step(action)


In [34]:
print("reward:", reward)
print("info keys:", info.keys())
print("info:", info)

reward: 3.3355588007393093
info keys: dict_keys(['time', 'rwd_dense', 'rwd_sparse', 'solved', 'done', 'obs_dict', 'visual_dict', 'proprio_dict', 'rwd_dict', 'state'])
info: {'time': 0.01, 'rwd_dense': 3.3355588007393093, 'rwd_sparse': False, 'solved': False, 'done': 0, 'obs_dict': {'time': array(0.01), 'pelvis_pos': array([ 2.0201e+00, -1.0802e-03,  9.5000e-01]), 'body_qpos': array([-4.2014e-01,  4.1108e-04, -5.1736e-03,  4.4879e-04, -3.5817e-03,
       -4.4112e-07,  1.9143e-05, -3.1711e-03, -8.8424e-04,  9.1849e-05,
       -1.9420e-04, -8.7721e-02,  1.3843e-04, -1.5749e-04, -1.0380e-03,
        1.4430e-04, -1.2248e-04, -1.2347e-03,  1.1688e-04, -1.1798e-04,
       -1.4396e-02,  5.1444e-03, -5.1316e-03,  1.4270e-02, -3.9553e-03,
        1.7212e-02,  3.0272e-03, -2.8441e-03, -1.7449e-02,  1.2466e-01,
        1.7207e-01,  7.0511e-01, -6.9755e-01, -3.7210e-01,  7.9305e-01,
       -1.2462e-01, -7.2743e-02, -7.5158e-01,  9.1993e-02,  7.3577e-01,
        5.5032e-02, -3.1576e-01,  6.8302e-01,

In [35]:
info['done']

0